# Initialize

In [ ]:
! pip install shap

import shap
import pandas as pd
import numpy as np 
import xgboost as xgb 
import os
import matplotlib.pylab as plt

from sklearn import metrics
from sklearn.model_selection import train_test_split 
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc, precision_score, recall_score, f1_score, make_scorer, log_loss
from sklearn.model_selection import cross_val_score, cross_validate, cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.inspection import PartialDependenceDisplay
from matplotlib.pylab import rcParams
from matplotlib import pyplot

%matplotlib inline

RAND_STATE = 30

def Performance(Model,Y,X,Yt,Xt,title):
    rcParams['figure.figsize'] = 4, 4
    # Perforamnce of the model
    fpr, tpr, _ = roc_curve(Y, Model.predict_proba(X)[:,1])
    AUC  = metrics.auc(fpr, tpr)
    fprt, tprt, _ = roc_curve(Yt, Model.predict_proba(Xt)[:,1])
    AUCt  = metrics.auc(fprt, tprt)
    plt.figure()
    plt.plot(fprt, tprt, label='AUC Train = %0.4f' % AUCt)
    plt.plot(fpr, tpr, label='AUC Test = %0.4f' % AUC)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('1 - Specificity')
    plt.ylabel('Sensitivity')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()


# define a function which will help us create XGBoost models and perform cross-validation
def modelfit(alg, dtrain, dtarget, xtest, ytest, cv_folds, estn, useTrainCV=True, early_stopping_rounds=50):
    predictors = [x for x in dtrain.columns]
    if useTrainCV:
        cv = StratifiedKFold(n_splits=cv_folds)
        xgb_param = alg.get_xgb_params()
        print(f'Using CV with {cv_folds} folds')
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtarget.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], folds=cv,
            metrics=['logloss','auc'], early_stopping_rounds=early_stopping_rounds)
        print("CV results:")
        print(f"  Train logloss = {np.mean(cvresult['train-logloss-mean']):.3f} ({np.mean(cvresult['train-logloss-std']):.3f})" \
              f"  Train AUC = {np.mean(cvresult['train-auc-mean']):.3f} ({np.mean(cvresult['train-auc-std']):.3f})")
        print(f"  Test logloss = {np.mean(cvresult['test-logloss-mean']):.3f} ({np.mean(cvresult['test-logloss-std']):.3f})" \
              f"  Test AUC = {np.mean(cvresult['test-auc-mean']):.3f} ({np.mean(cvresult['test-auc-std']):.3f})")
        if estn == 0:
          print(f"\nOptimum trees using XGB CV={cvresult.shape[0]}")
          alg.set_params(n_estimators=cvresult.shape[0]) # check the optimum number of trees using cv function of xgboost
        else:
          print(f"\nOptimum trees using XGB CV={cvresult.shape[0]}, currently set to {estn}")
          alg.set_params(n_estimators=estn)
 
    #Fit the algorithm on the data
    print('fitting...')
    alg.fit(dtrain[predictors], dtarget.values.ravel(), eval_set=[(dtrain, dtarget.values.ravel()), (xtest, ytest.values.ravel())], 
            eval_metric=['logloss'], early_stopping_rounds=early_stopping_rounds, verbose=0)
    print(f'Early stopping: best (score, iter, ntree limit) = {alg.best_score, alg.best_iteration, alg.best_ntree_limit}') 

    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors]) # i.e.: y_pred
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1] # i.e.: y_score
    dtest_predict = alg.predict(xtest)
    dtest_predprob = alg.predict_proba(xtest)[:,1]
        
    #Print model report:
    print(f"\n{'MODEL REPORT':26} {'train':10} | {'test':10}")
    print(f" {'Accuracy:':25} {metrics.accuracy_score(dtarget.values, dtrain_predictions):10.4f} | " \
          f"{metrics.accuracy_score(ytest.values, dtest_predict):10.4f}")
    print(f" {'Precision Score:':25} {metrics.precision_score(dtarget.values, dtrain_predictions):10.4f} | " \
          f"{metrics.precision_score(ytest.values, dtest_predict):10.4f}")
    print(f" {'Recall (sensitivity):':25} {metrics.recall_score(dtarget, dtrain_predictions):10.4f} | " \
          f"{metrics.recall_score(ytest, dtest_predict):10.4f}")
    print(f" {'F1 Score:':25} {metrics.f1_score(dtarget.values, dtrain_predictions):10.4f} | " \
          f"{metrics.f1_score(ytest.values, dtest_predict):10.4f}")
    print(f" {'AUC Score:':25} {metrics.roc_auc_score(dtarget, dtrain_predprob):10.4f} | " \
          f"{metrics.roc_auc_score(ytest, dtest_predprob):10.4f}")

    rcParams['figure.figsize'] = 12, 4
    feat_imp = pd.Series(alg.get_booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

# Data handling
load both databases, format (including one-hot encoding where appropriate), and merge

In [ ]:
df = pd.DataFrame(data=[]) # reset df

# fix eicu 
df = pd.read_csv('/eICU_all_ml.csv')
df = pd.concat([df.loc[:,:'currICUDay'],
                df.loc[:,'dr_uo'],
                df.loc[:,'age':'ethnicity'],
                df.loc[:,'congestive_heart_failure':]], axis=1)

df = df.loc[df['dr_uo'].notnull()].copy()
df['ethnicity'].replace({'African American':'Other', 'Hispanic':'Other', 'Asian':'Other', 'Native American':'Other',
                         'Other/Unknown':'Other'},inplace=True)
df.rename(columns={"ethnicity": "ethn", "gender": "sex"}, inplace=True)
df = pd.get_dummies(df, columns=['sex','ethn'])
print('eicu data formatted')

# fix mimic
df_ = pd.read_csv('/MIMIC_all_noimpute.csv')
df_ = df_[df_['dr_uo'].isnull()==False]
df_ = pd.concat([df_.loc[:,'dr_uo'],
                df_.loc[:,'age':'ethn'],
                df_.loc[:,'congestive_heart_failure':'depression'], 
                df_.loc[:,'s_Na':]], axis=1) # gender + ethn + elixhauser + other diuretics + DDI + labs

df_['sex'].replace({1: 'female', 0: 'male'},inplace=True)
df_['ethn'].replace({0: 'Caucasian', 1: 'Other', 2: 'Other', 3:'Other', 4: 'Other', 5: 'Other', 6: 'Other'},inplace=True)

df_ = pd.get_dummies(df_, columns=['ethn','sex'])
print("finished mimic data prep")

# short Cols

In [ ]:
# select variables to be included in analysis (abbr)
cols_1=['age', 'sex_female', 'ethn_Caucasian', 's_Cl', 's_K', 's_HCO3', 's_Cr', 's_Glu', 's_Ca', 's_Hct', 's_Plt', 's_WBC', 'avg_HR', 'avg_sBP', 'avg_RR', 'avg_TempC', 'avg_SpO2', 'congestive_heart_failure', 'diabetes_uncomplicated', 'renal_failure', 'liver_disease']
cols_2=['age', 'sex_female', 'ethn_Caucasian', 's_Cl', 's_K', 's_HCO3', 's_Cr', 's_Glu', 's_Ca', 's_Hgb', 's_Plt', 's_WBC', 'avg_HR', 'avg_sBP', 'avg_RR', 'avg_TempC', 'avg_SpO2', 'congestive_heart_failure', 'diabetes_uncomplicated', 'renal_failure', 'liver_disease']
print("columns complete")

# **Train/Test**

In [ ]:
# generate train and test datasets
# set test dataset to 20% of data-> 80/20 split = Pareto principle
# for models 1, 2, 4

X = df.loc[:,cols_1].copy()
y = df['dr_uo'].copy()

X__test = df_.loc[:,cols_1].copy()
y__test = df_['dr_uo'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y) 
print("Train and Test datasets created for models 1, 2, and 4... XGBoost ready")


# for model 3

X2 = df.loc[:,cols_2].copy()
y2 = df['dr_uo'].copy()

X2__test = df_.loc[:,cols_2].copy()
y2__test = df_['dr_uo'].copy()

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.20, stratify=y2) # set test dataset to 20% of data-> 80/20 split = Pareto principle
print("Train and Test datasets created for model 4... XGBoost ready")

# **Model 1**

In [ ]:
# build xgb models with hyperparameter settings

n_est = 413 # set to 0 to allow CV to pick

xgb_1 = xgb.XGBClassifier(objective='binary:logistic',
                     max_depth = 9,
                     min_child_weight = 5,
                     gamma = 9,
                     subsample = 0.9478124851219693,
                     colsample_bytree = 0.8067607006519033, 
                     reg_alpha = 0.9946342157271248,
                     reg_lambda = 1,
                     learning_rate = 0.1,
                     eval_metric='logloss',
                     missing=np.nan,
                     use_label_encoder=False) 

modelfit(xgb_1, X_train, y_train, X_test, y_test, 5, n_est) # last # = n_estimators, 2nd last = Kfolds for CV

Performance(Model=xgb_1,Y=y_test,X=X_test,Yt=y_train,Xt=X_train,title="Model 1 ROC")

pars=xgb_1.get_xgb_params()
print("\nfinal tuning:")
print(f"'max_depth':{pars['max_depth']}, 'min_child_weight':{pars['min_child_weight']}, 'gamma':{pars['gamma']}" \
      f", 'subsample':{pars['subsample']}, 'colsample_bytree':{pars['colsample_bytree']}, " \
      f" 'reg_alpha':{pars['reg_alpha']}, 'reg_lambda':{pars['reg_lambda']}")

# **Model 2**

In [ ]:
n_est = 304 # set to 0 to allow CV to pick

xgb_2 = xgb.XGBClassifier(objective='binary:logistic',
                     max_depth = 3,
                     min_child_weight = 2,
                     gamma = 4.516721893114058,
                     subsample = 0.8469450038636233,
                     colsample_bytree = 0.968174437977502, 
                     reg_alpha = 0.18678516647095939,
                     reg_lambda = 1,
                     learning_rate = 0.1,
                     eval_metric='logloss', 
                     missing=np.nan,
                     use_label_encoder=False) 

modelfit(xgb_2, X_train, y_train, X_test, y_test, 5, n_est) # last # = n_estimators, 2nd last = Kfolds for CV

Performance(Model=xgb_2,Y=y_test,X=X_test,Yt=y_train,Xt=X_train,title="Model 2 ROC")

pars=xgb_2.get_xgb_params()
print("\nfinal tuning:")
print(f"'max_depth':{pars['max_depth']}, 'min_child_weight':{pars['min_child_weight']}, 'gamma':{pars['gamma']}" \
      f", 'subsample':{pars['subsample']}, 'colsample_bytree':{pars['colsample_bytree']}, " \
      f" 'reg_alpha':{pars['reg_alpha']}, 'reg_lambda':{pars['reg_lambda']}")

# **Model 3**

In [ ]:
n_est = 157 # set to 0 to allow CV to pick

xgb__ran = xgb.XGBClassifier(objective='binary:logistic',
                     max_depth = 3,
                     min_child_weight = 1,
                     gamma = 4.316,
                     subsample = 0.8161,
                     colsample_bytree = 0.937, 
                     reg_alpha = 0.3988,
                     reg_lambda = 1,
                     learning_rate = 0.1,
                     eval_metric='logloss', 
                     missing=np.nan,
                     use_label_encoder=False) 

modelfit(xgb_3, X2_train, y2_train, X2_test, y2_test, 5, n_est) # last # = n_estimators, 2nd last = Kfolds for CV

Performance(Model=xgb_3,Y=y2_test,X=X2_test,Yt=y2_train,Xt=X2_train,title="Model 3 ROC")

pars=xgb_3.get_xgb_params()
print("\nfinal tuning:")
print(f"'max_depth':{pars['max_depth']}, 'min_child_weight':{pars['min_child_weight']}, 'gamma':{pars['gamma']}" \
      f", 'subsample':{pars['subsample']}, 'colsample_bytree':{pars['colsample_bytree']}, " \
      f" 'reg_alpha':{pars['reg_alpha']}, 'reg_lambda':{pars['reg_lambda']}")

# **Model 4**

In [ ]:
n_est = 196 # set to 0 to allow CV to pick

xgb__roc = xgb.XGBClassifier(objective='binary:logistic',
                     max_depth = 9,
                     min_child_weight = 1,
                     gamma = 4.474622122766906,
                     subsample = 0.9880695217296125,
                     colsample_bytree = 0.8577812370816866, 
                     reg_alpha = 7.919701369390835,
                     reg_lambda = 1,
                     learning_rate = 0.1,
                     eval_metric='logloss', 
                     missing=np.nan,
                     use_label_encoder=False) 

modelfit(xgb_4, X_train, y_train, X_test, y_test, 5, n_est) # last # = n_estimators, 2nd last = Kfolds for CV

Performance(Model=xgb_4,Y=y_test,X=X_test,Yt=y_train,Xt=X_train,title="Model 4 ROC")

pars=xgb_4.get_xgb_params()
print("\nfinal tuning:")
print(f"'max_depth':{pars['max_depth']}, 'min_child_weight':{pars['min_child_weight']}, 'gamma':{pars['gamma']}" \
      f", 'subsample':{pars['subsample']}, 'colsample_bytree':{pars['colsample_bytree']}, " \
      f" 'reg_alpha':{pars['reg_alpha']}, 'reg_lambda':{pars['reg_lambda']}")

# SHAP, partial dependence

In [ ]:
# set model
X_model = xgb_1
X_full = X

In [ ]:
# plot feature importance plots of gain and weight

xgb.plot_importance(X_model, title='Feature importance: Gain', grid=False,
                    importance_type='gain', show_values=False)

xgb.plot_importance(X_model, title='Feature importance: Weight', grid=False,
                    importance_type='weight', show_values=False)

In [ ]:
# generate SHAP summary plot

explainer = shap.TreeExplainer(X_model)
shap_values = explainer.shap_values(X_full)
shap.summary_plot(shap_values, X_full, color='white')

In [ ]:
# generate partial dependence plot of 4 most important variables

pdp = PartialDependenceDisplay.from_estimator(X_model, X_full, ["s_Cr", "avg_sBP", "s_Cl", "age"], n_cols=4)

#***Ensemble Model***

In [ ]:
# create function to return probability based on threshold
def adj_prob(y_pred, t):
  if t == 0.5: 
      return [y_pred][0]
  else: 
      return [0.4 if (y >= 0.5 and y < t) else y for y in y_pred]

# set number of models
num_models = 4.0 
print(f'{num_models} models set')

In [ ]:
# train datasets
ypred_xgb_1_train = adj_prob(xgb_1.predict_proba(X_train)[:,1], t=0.5)
ypred_xgb_2_train = adj_prob(xgb_2.predict_proba(X_train)[:,1], t=0.5)
ypred_xgb_3_train = adj_prob(xgb_3.predict_proba(X2_train)[:,1], t=0.5)
ypred_xgb_4_train = adj_prob(xgb_4.predict_proba(X_train)[:,1], t=0.5)

# test datasets
ypred_xgb_1_test = adj_prob(xgb_1.predict_proba(X_test)[:,1], t=0.5)
ypred_xgb_2_test = adj_prob(xgb_2.predict_proba(X_test)[:,1], t=0.5)
ypred_xgb_3_test = adj_prob(xgb_3.predict_proba(X2_test)[:,1], t=0.5)
ypred_xgb_4_test = adj_prob(xgb_4.predict_proba(X_test)[:,1], t=0.5)

# validate dataset
ypred_xgb_1_val = adj_prob(xgb_1.predict_proba(X__test)[:,1], t=0.5)
ypred_xgb_2_val = adj_prob(xgb_2.predict_proba(X__test)[:,1], t=0.5)
ypred_xgb_3_val = adj_prob(xgb_3.predict_proba(X2__test)[:,1], t=0.5)
ypred_xgb_4_val = adj_prob(xgb_4.predict_proba(X__test)[:,1], t=0.5)


# geometric means
ypredprob_geo_mean_train = [(aa*bb*cc*dd)**(1/num_models) for aa, bb, cc, dd in zip(ypred_xgb_1_train, ypred_xgb_2_train, ypred_xgb_3_train, ypred_xgb_4_train)]
ypredprob_geo_mean_test = [(aa*bb*cc*dd)**(1/num_models) for aa, bb, cc, dd in zip(ypred_xgb_1_test, ypred_xgb_2_test, ypred_xgb_3_test, ypred_xgb_4_test)]
ypredprob_geo_mean_val = [(aa*bb*cc*dd)**(1/num_models) for aa, bb, cc, dd in zip(ypred_xgb_1_val, ypred_xgb_2_val, ypred_xgb_3_val, ypred_xgb_4_val)]
ypred_geo_train = [1 if y >= 0.5 else 0 for y in ypredprob_geo_mean_train]
ypred_geo_test = [1 if y >= 0.5 else 0 for y in ypredprob_geo_mean_test]
ypred_geo_val = [1 if y >= 0.5 else 0 for y in ypredprob_geo_mean_val]


# validate optimized for specificity
THRESH = 0.785
ypred_val_sp, ypred_prob_val_sp = [1 if y >= THRESH else 0 for y in ypredprob_geo_mean_val], [0.49 if (y >= 0.5 and y < THRESH) else y for y in ypredprob_geo_mean_val] 


# generate AUROC
fpr_train, tpr_train, _ = roc_curve(y_train.values, ypredprob_geo_mean_train)
fpr_test, tpr_test, _ = roc_curve(y_test.values, ypredprob_geo_mean_test)
fpr_val, tpr_val, _ = roc_curve(y__test.values, ypredprob_geo_mean_val)
fpr_valsp, tpr_valsp, _ = roc_curve(y__test.values, ypred_prob_val_sp)

AUC_train = metrics.auc(fpr_train, tpr_train)
AUC_test = metrics.auc(fpr_test, tpr_test)
AUC_val = metrics.auc(fpr_val, tpr_val)
AUC_valsp = metrics.auc(fpr_valsp, tpr_valsp)

# Plot all ROC curves
plt.figure()
plt.plot(fpr_train, tpr_train, label= ' Train, AUC = %0.4f' % AUC_train)
plt.plot(fpr_test, tpr_test, label= ' Test, AUC = %0.4f' % AUC_test)
plt.plot(fpr_val, tpr_val, label= ' Validate, AUC = %0.4f' % AUC_val)
plt.plot(fpr_valsp, tpr_valsp, label= f" Validate, AUC = {AUC_valsp:0.4f} (specificity optimized)")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

# generate stats
sn, sp, ppv, npv, pl, nl = [0 for i in range(4)] , [0 for i in range(4)], [0 for i in range(4)], [0 for i in range(4)], [0 for i in range(4)], [0 for i in range(4)]
for i in range(0,4): 
  if i == 0:
    txt = "Train"
    cm = confusion_matrix(y_train, ypred_geo_train) 
    ModelReport(txt, y_train, ypred_geo_train, ypredprob_geo_mean_train)
  if i == 1:
    txt = "Test"
    cm = confusion_matrix(y_test, ypred_geo_test) 
    ModelReport(txt, y_test, ypred_geo_test, ypredprob_geo_mean_test)
  if i == 2:
    txt = "Validate"
    cm = confusion_matrix(y__test, ypred_geo_val) 
    ModelReport(txt, y__test, ypred_geo_val, ypredprob_geo_mean_val)
  if i == 3:
    txt = "Validate (Spec optimized)"
    cm = confusion_matrix(y__test, ypred_val_sp) 
    ModelReport(txt, y__test, ypred_val_sp, ypred_prob_val_sp)
  
  sn[i] = cm[1,1]/(cm[1,1]+cm[1,0]) # Sens = TP / (TP + FN)
  sp[i] = cm[0,0]/(cm[0,0]+cm[0,1]) # Spec = TN / (FP + TN)
  ppv[i] = cm[1,1]/(cm[1,1]+cm[0,1]) # PPV = TP / (TP + FP)
  npv[i] = cm[0,0]/(cm[0,0]+cm[1,0])# NPV = TN / (FN + TN)
  pl[i] = sn[i]/(1-sp[i])
  nl[i] = (1-sn[i])/sp[i]

  print(f"\nCharacteristics, {txt}")
  print(f"  sens: {sn[i]:0.4f}")
  print(f"  spec: {sp[i]:0.4f}")
  print(f"   ppv: {ppv[i]:0.4f}")
  print(f"   npv: {npv[i]:0.4f}")
  print(f"   LR+: {pl[i]:0.4f}")
  print(f"   LR-: {nl[i]:0.4f}")
  print(f"\n")
  